In [37]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from functools import partial

In [2]:
# initializers in keras
[name for name in dir(keras.initializers) if not name.startswith("_")]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'HeNormal',
 'HeUniform',
 'Identity',
 'Initializer',
 'LecunNormal',
 'LecunUniform',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'random_normal',
 'random_uniform',
 'serialize',
 'truncated_normal',
 'variance_scaling',
 'zeros']

In [3]:
# dataset 
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [4]:
tf.random.set_seed(42)
np.random.seed(42)

In [5]:
# custom intializers
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',distribution='uniform')

# leaky relu
leaky_relu = keras.layers.LeakyReLU(alpha=0.2)

# selu
layer = keras.layers.Dense(10, activation="selu",kernel_initializer="lecun_normal")

In [6]:
# Model with HE initlialization and Leaky Relu activation

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal"),
                                 keras.layers.LeakyReLU(),
                                 keras.layers.Dense(100, kernel_initializer="he_normal"),
                                 keras.layers.LeakyReLU(),
                                 keras.layers.Dense(10, activation="softmax")])

In [7]:
model.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-3),metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))              

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7955 - accuracy: 0.7362 - val_loss: 0.7130 - val_accuracy: 0.7656
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7898
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8066
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5832 - accuracy: 0.8075 - val_loss: 0.5582 - val_accuracy: 0.8198
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5553 - accuracy: 0.8157 - val_loss: 0.5350 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5338 - accuracy: 0.8225 - val_loss: 0.5156 - val_accuracy:

In [8]:
tf.random.set_seed(42)
np.random.seed(42)

# Model with Lecun Normal initlialization and Leaky Relu activation

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, activation="selu",kernel_initializer="lecun_normal"),
                                 keras.layers.Dense(100, activation="selu",kernel_initializer="lecun_normal"),
                                 keras.layers.Dense(10, activation="softmax")])

# normalizing input
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

# model complile
model.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-3),metrics=["accuracy"])

# train the model
history = model.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_valid_scaled, y_valid))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7083 - accuracy: 0.7570 - val_loss: 0.5345 - val_accuracy: 0.8156
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5048 - accuracy: 0.8244 - val_loss: 0.4741 - val_accuracy: 0.8342
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4619 - accuracy: 0.8385 - val_loss: 0.4510 - val_accuracy: 0.8430
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4374 - accuracy: 0.8478 - val_loss: 0.4327 - val_accuracy: 0.8486
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4207 - accuracy: 0.8538 - val_loss: 0.4216 - val_accuracy: 0.8538
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4074 - accuracy: 0.8571 - val_loss: 0.4129 - val_accuracy: 0.8558
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3970 - accuracy: 0.8611 - val_loss: 0.4044 - val_accuracy:

In [9]:
# Model with Prelu


tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal"),
                                 keras.layers.PReLU(),
                                 keras.layers.Dense(100, kernel_initializer="he_normal"),
                                 keras.layers.PReLU(),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-3),metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))              

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 1.3461 - accuracy: 0.6209 - val_loss: 0.9255 - val_accuracy: 0.7186
Epoch 2/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.8197 - accuracy: 0.7356 - val_loss: 0.7305 - val_accuracy: 0.7628
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6966 - accuracy: 0.7693 - val_loss: 0.6565 - val_accuracy: 0.7878
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6331 - accuracy: 0.7909 - val_loss: 0.6004 - val_accuracy: 0.8046
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5917 - accuracy: 0.8057 - val_loss: 0.5656 - val_accuracy: 0.8182
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5618 - accuracy: 0.8136 - val_loss: 0.5406 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.5390 - accuracy: 0.8206 - val_loss: 0.5196 - val_accuracy:

In [10]:
# Relu activation
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(100, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-3),metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))   

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 1.3468 - accuracy: 0.6209 - val_loss: 0.9259 - val_accuracy: 0.7186
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.8200 - accuracy: 0.7354 - val_loss: 0.7308 - val_accuracy: 0.7626
Epoch 3/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.6968 - accuracy: 0.7694 - val_loss: 0.6566 - val_accuracy: 0.7880
Epoch 4/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.6333 - accuracy: 0.7911 - val_loss: 0.6005 - val_accuracy: 0.8042
Epoch 5/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5918 - accuracy: 0.8054 - val_loss: 0.5658 - val_accuracy: 0.8180
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5619 - accuracy: 0.8135 - val_loss: 0.5407 - val_accuracy: 0.8232
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5391 - accuracy: 0.8204 - val_loss: 0.5197 - val_accuracy:

In [11]:
## Training with Batch Normalization after the activation function
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-3),metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid)) 

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.8727 - accuracy: 0.7073 - val_loss: 0.6013 - val_accuracy: 0.7950
Epoch 2/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.6018 - accuracy: 0.7933 - val_loss: 0.5190 - val_accuracy: 0.8218
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5466 - accuracy: 0.8109 - val_loss: 0.4829 - val_accuracy: 0.8352
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5108 - accuracy: 0.8231 - val_loss: 0.4597 - val_accuracy: 0.8428
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4871 - accuracy: 0.8296 - val_loss: 0.4434 - val_accuracy: 0.8462
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4712 - accuracy: 0.8350 - val_loss: 0.4309 - val_accuracy: 0.8530
Epoch 7/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4591 - accuracy: 0.8403 - val_loss: 0.4199 - val_accuracy

In [12]:
# To sum up, four parameter vectors are learned in each batch-normalized
# layer: gamma (the ouput scale vector) and beta (the output offset vector) are learned through
# regular backpropagation, and mean (the final input mean vector), and variance (the final input
# standard deviation vector) are estimated using an exponential moving average. which are used for prediction

[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [13]:
## Training with Batch Normalization before the activation function
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Activation("elu"),
                                 keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
                                 keras.layers.Activation("elu"),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-3),metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid)) 

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.8720 - accuracy: 0.7082 - val_loss: 0.6038 - val_accuracy: 0.7956
Epoch 2/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.6055 - accuracy: 0.7924 - val_loss: 0.5222 - val_accuracy: 0.8212
Epoch 3/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5505 - accuracy: 0.8097 - val_loss: 0.4865 - val_accuracy: 0.8344
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5155 - accuracy: 0.8217 - val_loss: 0.4633 - val_accuracy: 0.8402
Epoch 5/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4928 - accuracy: 0.8285 - val_loss: 0.4472 - val_accuracy: 0.8456
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4768 - accuracy: 0.8341 - val_loss: 0.4352 - val_accuracy: 0.8514
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4649 - accuracy: 0.8382 - val_loss: 0.4243 - val_accuracy:

In [14]:
# graddient clipping with clipvalue
optimizer = keras.optimizers.SGD(lr=1e-3,clipvalue=1.0)

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(100, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))   

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 1.3468 - accuracy: 0.6209 - val_loss: 0.9259 - val_accuracy: 0.7186
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.8200 - accuracy: 0.7354 - val_loss: 0.7308 - val_accuracy: 0.7626
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6968 - accuracy: 0.7694 - val_loss: 0.6566 - val_accuracy: 0.7880
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6333 - accuracy: 0.7911 - val_loss: 0.6005 - val_accuracy: 0.8042
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5918 - accuracy: 0.8054 - val_loss: 0.5658 - val_accuracy: 0.8180
Epoch 6/10
1719/1719 [==============================] - 4s 3ms/step - loss: 0.5619 - accuracy: 0.8135 - val_loss: 0.5407 - val_accuracy: 0.8232
Epoch 7/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.5391 - accuracy: 0.8204 - val_loss: 0.5197 - val_accuracy:

In [15]:
# graddient clipping with clipnorm
optimizer = keras.optimizers.SGD(lr=1e-3,clipnorm=1.0)

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(100, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))  

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 1.5560 - accuracy: 0.5628 - val_loss: 1.0931 - val_accuracy: 0.6886
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.9391 - accuracy: 0.7088 - val_loss: 0.8213 - val_accuracy: 0.7394
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7755 - accuracy: 0.7452 - val_loss: 0.7210 - val_accuracy: 0.7678
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6991 - accuracy: 0.7684 - val_loss: 0.6610 - val_accuracy: 0.7828
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6513 - accuracy: 0.7841 - val_loss: 0.6229 - val_accuracy: 0.7978
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6171 - accuracy: 0.7951 - val_loss: 0.5937 - val_accuracy: 0.8048
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5908 - accuracy: 0.8042 - val_loss: 0.5696 - val_accuracy:

In [16]:
## Resusing Mdelling - Create Base Model dataset


def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [17]:
# Train Base Model A
tf.random.set_seed(42)
np.random.seed(42)

model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

model_A.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-3),metrics=["accuracy"])
                

history = model_A.fit(X_train_A, y_train_A, epochs=20,validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.5926 - accuracy: 0.8103 - val_loss: 0.3895 - val_accuracy: 0.8665
Epoch 2/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3523 - accuracy: 0.8784 - val_loss: 0.3287 - val_accuracy: 0.8824
Epoch 3/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3171 - accuracy: 0.8896 - val_loss: 0.3014 - val_accuracy: 0.8994
Epoch 4/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2974 - accuracy: 0.8976 - val_loss: 0.2894 - val_accuracy: 0.9021
Epoch 5/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2836 - accuracy: 0.9020 - val_loss: 0.2775 - val_accuracy: 0.9061
Epoch 6/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.2730 - accuracy: 0.9059 - val_loss: 0.2735 - val_accuracy: 0.9063
Epoch 7/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.2642 - accuracy: 0.9094 - val_loss: 0.2721 - val_accuracy:

In [18]:
model_A.save("my_model_A.h5")

In [19]:
### Training Model B from scratch
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))

for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))

model_B.add(keras.layers.Dense(1, activation="sigmoid"))

model_B.compile(loss="binary_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-3),metrics=["accuracy"])

history = model_B.fit(X_train_B, y_train_B, epochs=20,validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 0s 38ms/step - loss: 0.9573 - accuracy: 0.4650 - val_loss: 0.6314 - val_accuracy: 0.6004
Epoch 2/20
7/7 [==============================] - 0s 22ms/step - loss: 0.5692 - accuracy: 0.7450 - val_loss: 0.4784 - val_accuracy: 0.8529
Epoch 3/20
7/7 [==============================] - 0s 18ms/step - loss: 0.4503 - accuracy: 0.8650 - val_loss: 0.4102 - val_accuracy: 0.8945
Epoch 4/20
7/7 [==============================] - 0s 16ms/step - loss: 0.3879 - accuracy: 0.8950 - val_loss: 0.3647 - val_accuracy: 0.9178
Epoch 5/20
7/7 [==============================] - 0s 20ms/step - loss: 0.3435 - accuracy: 0.9250 - val_loss: 0.3300 - val_accuracy: 0.9320
Epoch 6/20
7/7 [==============================] - 0s 22ms/step - loss: 0.3081 - accuracy: 0.9300 - val_loss: 0.3019 - val_accuracy: 0.9402
Epoch 7/20
7/7 [==============================] - 0s 20ms/step - loss: 0.2800 - accuracy: 0.9350 - val_loss: 0.2804 - val_accuracy: 0.9422
Epoch 8/20
7/7 [===========

In [20]:
# Training Model B by reusing layers from Model A


model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))


# clone Model A as training will modify the model paramters
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-3),metrics=["accuracy"])

In [21]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",optimizer=keras.optimizers.SGD(lr=1e-3),metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 47ms/step - loss: 0.5813 - accuracy: 0.6500 - val_loss: 0.5860 - val_accuracy: 0.6318
Epoch 2/4
7/7 [==============================] - 0s 16ms/step - loss: 0.5446 - accuracy: 0.6700 - val_loss: 0.5482 - val_accuracy: 0.6704
Epoch 3/4
7/7 [==============================] - 0s 18ms/step - loss: 0.5075 - accuracy: 0.7300 - val_loss: 0.5160 - val_accuracy: 0.7069
Epoch 4/4
7/7 [==============================] - 0s 16ms/step - loss: 0.4758 - accuracy: 0.7500 - val_loss: 0.4871 - val_accuracy: 0.7292
Epoch 1/16
7/7 [==============================] - 0s 42ms/step - loss: 0.3968 - accuracy: 0.8150 - val_loss: 0.3469 - val_accuracy: 0.8631
Epoch 2/16
7/7 [==============================] - 0s 20ms/step - loss: 0.2802 - accuracy: 0.9350 - val_loss: 0.2609 - val_accuracy: 0.9249
Epoch 3/16
7/7 [==============================] - 0s 18ms/step - loss: 0.2087 - accuracy: 0.9650 - val_loss: 0.2115 - val_accuracy: 0.9554
Epoch 4/16
7/7 [===============

In [22]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.1408 - accuracy: 0.9705


[0.1408407837152481, 0.9704999923706055]

In [23]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0683 - accuracy: 0.9930


[0.06832191348075867, 0.9929999709129333]

In [24]:
# momentum optimization
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(100, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.7049 - accuracy: 0.7683 - val_loss: 0.5035 - val_accuracy: 0.8300
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4798 - accuracy: 0.8327 - val_loss: 0.4546 - val_accuracy: 0.8476
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4362 - accuracy: 0.8470 - val_loss: 0.4152 - val_accuracy: 0.8582
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4097 - accuracy: 0.8569 - val_loss: 0.4004 - val_accuracy: 0.8622
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3921 - accuracy: 0.8633 - val_loss: 0.3791 - val_accuracy: 0.8694
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3771 - accuracy: 0.8668 - val_loss: 0.3684 - val_accuracy: 0.8716
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3649 - accuracy: 0.8714 - val_loss: 0.3593 - val_accuracy:

In [25]:
# Nesterov Accelerated Gradient
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(100, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7002 - accuracy: 0.7699 - val_loss: 0.4985 - val_accuracy: 0.8320
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4752 - accuracy: 0.8350 - val_loss: 0.4426 - val_accuracy: 0.8518
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4326 - accuracy: 0.8483 - val_loss: 0.4204 - val_accuracy: 0.8522
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4067 - accuracy: 0.8577 - val_loss: 0.4017 - val_accuracy: 0.8624
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3893 - accuracy: 0.8642 - val_loss: 0.3754 - val_accuracy: 0.8706
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3741 - accuracy: 0.8681 - val_loss: 0.3676 - val_accuracy: 0.8720
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3623 - accuracy: 0.8722 - val_loss: 0.3585 - val_accuracy:

In [26]:
# Adagrad optimization
optimizer = keras.optimizers.Adagrad(lr=0.001)

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(100, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 1.0012 - accuracy: 0.6995 - val_loss: 0.7025 - val_accuracy: 0.7744
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6501 - accuracy: 0.7871 - val_loss: 0.5943 - val_accuracy: 0.8102
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5768 - accuracy: 0.8124 - val_loss: 0.5520 - val_accuracy: 0.8220
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5382 - accuracy: 0.8237 - val_loss: 0.5159 - val_accuracy: 0.8342
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5133 - accuracy: 0.8303 - val_loss: 0.4960 - val_accuracy: 0.8388
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4951 - accuracy: 0.8336 - val_loss: 0.4812 - val_accuracy: 0.8426
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4809 - accuracy: 0.8378 - val_loss: 0.4682 - val_accuracy:

In [27]:
# RMSProp
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(100, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5038 - accuracy: 0.8167 - val_loss: 0.4570 - val_accuracy: 0.8396
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3921 - accuracy: 0.8612 - val_loss: 0.4112 - val_accuracy: 0.8598
Epoch 3/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3717 - accuracy: 0.8717 - val_loss: 0.6163 - val_accuracy: 0.8122
Epoch 4/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3563 - accuracy: 0.8754 - val_loss: 0.3640 - val_accuracy: 0.8828
Epoch 5/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3492 - accuracy: 0.8812 - val_loss: 0.4623 - val_accuracy: 0.8738
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3381 - accuracy: 0.8848 - val_loss: 0.4417 - val_accuracy: 0.8772
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3308 - accuracy: 0.8877 - val_loss: 0.4780 - val_accuracy:

In [28]:
# Adam Optimization
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(100, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4862 - accuracy: 0.8246 - val_loss: 0.3643 - val_accuracy: 0.8662
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3658 - accuracy: 0.8662 - val_loss: 0.4018 - val_accuracy: 0.8564
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3307 - accuracy: 0.8779 - val_loss: 0.3156 - val_accuracy: 0.8870
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3050 - accuracy: 0.8856 - val_loss: 0.3063 - val_accuracy: 0.8892
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2884 - accuracy: 0.8921 - val_loss: 0.3086 - val_accuracy: 0.8838
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2717 - accuracy: 0.8993 - val_loss: 0.3206 - val_accuracy: 0.8872
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2623 - accuracy: 0.9017 - val_loss: 0.3099 - val_accuracy:

In [29]:
# Admax Optimization
optimizer = keras.optimizers.Adamax(lr=0.001, beta_1=0.9, beta_2=0.999)

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(100, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5132 - accuracy: 0.8213 - val_loss: 0.3983 - val_accuracy: 0.8636
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3856 - accuracy: 0.8628 - val_loss: 0.3873 - val_accuracy: 0.8630
Epoch 3/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3454 - accuracy: 0.8739 - val_loss: 0.3376 - val_accuracy: 0.8792
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3174 - accuracy: 0.8848 - val_loss: 0.3133 - val_accuracy: 0.8838
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2992 - accuracy: 0.8908 - val_loss: 0.3070 - val_accuracy: 0.8906
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2844 - accuracy: 0.8963 - val_loss: 0.3125 - val_accuracy: 0.8866
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2717 - accuracy: 0.9006 - val_loss: 0.2948 - val_accuracy:

In [30]:
# Nadam Optimization

optimizer = keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999)

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(100, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))


Epoch 1/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4721 - accuracy: 0.8289 - val_loss: 0.3726 - val_accuracy: 0.8640
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3503 - accuracy: 0.8725 - val_loss: 0.3294 - val_accuracy: 0.8796
Epoch 3/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3176 - accuracy: 0.8829 - val_loss: 0.3534 - val_accuracy: 0.8704
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2922 - accuracy: 0.8916 - val_loss: 0.3228 - val_accuracy: 0.8880
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2773 - accuracy: 0.8958 - val_loss: 0.3069 - val_accuracy: 0.8870
Epoch 6/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2636 - accuracy: 0.9025 - val_loss: 0.3057 - val_accuracy: 0.8938
Epoch 7/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2519 - accuracy: 0.9054 - val_loss: 0.3097 - val_accuracy:

In [31]:
# Learning Rate Schedule - Power Scheduling
"""

lr = lr0 / (1 + steps / s)**c

    Keras uses c=1 and s = 1 / decay

"""

optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(100, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7021 - accuracy: 0.7700 - val_loss: 0.5205 - val_accuracy: 0.8262
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4876 - accuracy: 0.8297 - val_loss: 0.4415 - val_accuracy: 0.8524
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4450 - accuracy: 0.8440 - val_loss: 0.5029 - val_accuracy: 0.8124
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4214 - accuracy: 0.8531 - val_loss: 0.4074 - val_accuracy: 0.8622
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4060 - accuracy: 0.8584 - val_loss: 0.3912 - val_accuracy: 0.8656
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3914 - accuracy: 0.8626 - val_loss: 0.3863 - val_accuracy: 0.8704
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3820 - accuracy: 0.8659 - val_loss: 0.3821 - val_accuracy:

In [32]:
## Learning Schedule - Exponential decay

def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(100, kernel_initializer="he_normal",activation='relu'),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",optimizer='nadam',metrics=["accuracy"])


lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train_scaled, y_train, epochs=10,validation_data=(X_valid_scaled, y_valid),
                    callbacks=[lr_scheduler])

Epoch 1/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.7730 - accuracy: 0.7561 - val_loss: 0.6975 - val_accuracy: 0.7358
Epoch 2/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.6635 - accuracy: 0.7844 - val_loss: 0.6512 - val_accuracy: 0.8108
Epoch 3/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5356 - accuracy: 0.8313 - val_loss: 0.5087 - val_accuracy: 0.8396
Epoch 4/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4658 - accuracy: 0.8526 - val_loss: 0.4667 - val_accuracy: 0.8576
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4450 - accuracy: 0.8607 - val_loss: 0.5446 - val_accuracy: 0.8268
Epoch 6/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5405 - accuracy: 0.8315 - val_loss: 0.5677 - val_accuracy: 0.8332
Epoch 7/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4756 - accuracy: 0.8444 - val_loss: 0.5133 - val_accuracy:

In [33]:
# exponential decay updating the learning rate at the end of iteration instead of batch

K = keras.backend

class ExponentialDecay(keras.callbacks.Callback):
    def __init__(self, s=40000):
        super().__init__()
        self.s = s

    def on_batch_begin(self, batch, logs=None):
        # Note: the `batch` argument is reset at each epoch
        lr = K.get_value(self.model.optimizer.lr)
        K.set_value(self.model.optimizer.lr, lr * 0.1**(1 / s))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
                                 keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
                                 keras.layers.Dense(10, activation="softmax")])

lr0 = 0.01
optimizer = keras.optimizers.Nadam(lr=lr0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 10

s = 20 * len(X_train) // 32 # number of steps in 20 epochs (batch size = 32)

exp_decay = ExponentialDecay(s)

history = model.fit(X_train_scaled, y_train, epochs=n_epochs,validation_data=(X_valid_scaled, y_valid),
                    callbacks=[exp_decay])

Epoch 1/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.8340 - accuracy: 0.7586 - val_loss: 1.0441 - val_accuracy: 0.7446
Epoch 2/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.6719 - accuracy: 0.7965 - val_loss: 0.5452 - val_accuracy: 0.8212
Epoch 3/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.5808 - accuracy: 0.8197 - val_loss: 0.6421 - val_accuracy: 0.8114
Epoch 4/10
1719/1719 [==============================] - 9s 6ms/step - loss: 0.5488 - accuracy: 0.8331 - val_loss: 0.5430 - val_accuracy: 0.8292
Epoch 5/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4959 - accuracy: 0.8451 - val_loss: 0.5617 - val_accuracy: 0.8604
Epoch 6/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4552 - accuracy: 0.8566 - val_loss: 0.4542 - val_accuracy: 0.8492
Epoch 7/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4134 - accuracy: 0.8695 - val_loss: 0.4526 - val_accu

In [34]:
# Piece Wise constant learning rate
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.00
    
    
lr_scheduler = keras.callbacks.LearningRateScheduler(piecewise_constant_fn)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
                                 keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
                                 keras.layers.Dense(10, activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 10
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,validation_data=(X_valid_scaled, y_valid),
                    callbacks=[lr_scheduler])

Epoch 1/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.8725 - accuracy: 0.7526 - val_loss: 0.7744 - val_accuracy: 0.7876
Epoch 2/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.9104 - accuracy: 0.7297 - val_loss: 0.8860 - val_accuracy: 0.6904
Epoch 3/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.8078 - accuracy: 0.7627 - val_loss: 1.2252 - val_accuracy: 0.6354
Epoch 4/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.7851 - accuracy: 0.7677 - val_loss: 1.2156 - val_accuracy: 0.5210
Epoch 5/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.8788 - accuracy: 0.7257 - val_loss: 0.7876 - val_accuracy: 0.7928
Epoch 6/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5332 - accuracy: 0.8467 - val_loss: 0.6334 - val_accuracy: 0.8468
Epoch 7/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4864 - accuracy: 0.8589 - val_loss: 0.6823 - val_accuracy

In [35]:
# Performance Sceduling - Chnage learning rate if validation best is not improved
tf.random.set_seed(42)
np.random.seed(42)

# Reduce learning rate when a metric has stopped improving.
# factor: factor by which the learning rate will be reduced. new_lr = lr * factor
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
                                 keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
                                 keras.layers.Dense(10, activation="softmax")])

optimizer = keras.optimizers.SGD(lr=0.02, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

n_epochs = 10
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,validation_data=(X_valid_scaled, y_valid),
                    callbacks=[lr_scheduler])

Epoch 1/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5908 - accuracy: 0.8070 - val_loss: 0.4503 - val_accuracy: 0.8514
Epoch 2/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.4944 - accuracy: 0.8413 - val_loss: 0.6110 - val_accuracy: 0.8238
Epoch 3/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5178 - accuracy: 0.8415 - val_loss: 0.6581 - val_accuracy: 0.8238
Epoch 4/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5240 - accuracy: 0.8454 - val_loss: 0.5332 - val_accuracy: 0.8482
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5020 - accuracy: 0.8515 - val_loss: 0.4589 - val_accuracy: 0.8566
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.5123 - accuracy: 0.8539 - val_loss: 0.6514 - val_accuracy: 0.8452
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3033 - accuracy: 0.8945 - val_loss: 0.3940 - val_accuracy:

In [36]:
# learning rate schedule easy way 

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
                                 keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
                                 keras.layers.Dense(10, activation="softmax")])


# This approach updates the learning rate at each step rather than at each epoch
s = 20 * len(X_train) // 32 # number of steps in 20 epochs (batch size = 32)

# A LearningRateSchedule that uses an exponential decay schedule.
# initial_learning_rate * decay_rate ^ (step / decay_steps)
learning_rate = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate= 0.01,decay_steps= s, decay_rate=0.1)

# optimizer with learning rate
optimizer = keras.optimizers.SGD(learning_rate)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

n_epochs = 10

history = model.fit(X_train_scaled, y_train, epochs=n_epochs,validation_data=(X_valid_scaled, y_valid))

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4893 - accuracy: 0.8275 - val_loss: 0.4093 - val_accuracy: 0.8598
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3819 - accuracy: 0.8652 - val_loss: 0.3739 - val_accuracy: 0.8698
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3486 - accuracy: 0.8764 - val_loss: 0.3730 - val_accuracy: 0.8686
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3263 - accuracy: 0.8837 - val_loss: 0.3495 - val_accuracy: 0.8802
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3103 - accuracy: 0.8896 - val_loss: 0.3431 - val_accuracy: 0.8800
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2957 - accuracy: 0.8952 - val_loss: 0.3417 - val_accuracy: 0.8810
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2853 - accuracy: 0.8987 - val_loss: 0.3355 - val_accuracy:

In [38]:
#### L1 & L2 Regularization



# partial function to create dense layer so as to avoid repeating code 
RegularizedDense = partial(keras.layers.Dense, activation="elu", kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))



model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 RegularizedDense(300),
                                 RegularizedDense(100),
                                 RegularizedDense(10, activation="softmax",kernel_initializer="glorot_uniform")])


optimizer = keras.optimizers.SGD()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

n_epochs = 3
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,validation_data=(X_valid_scaled, y_valid))

Epoch 1/3
1719/1719 [==============================] - 6s 3ms/step - loss: 6.4569 - accuracy: 0.8212 - val_loss: 4.6113 - val_accuracy: 0.8582
Epoch 2/3
1719/1719 [==============================] - 6s 3ms/step - loss: 3.5158 - accuracy: 0.8526 - val_loss: 2.6297 - val_accuracy: 0.8582
Epoch 3/3
1719/1719 [==============================] - 6s 3ms/step - loss: 2.0910 - accuracy: 0.8543 - val_loss: 1.6578 - val_accuracy: 0.8540


In [39]:
### Drop out regularization

model = keras.models.Sequential([keras.layers.Flatten(input_shape=[28, 28]),
                                 keras.layers.Dropout(rate=0.2),
                                 keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
                                 keras.layers.Dropout(rate=0.2),
                                 keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
                                 keras.layers.Dropout(rate=0.2),
                                 keras.layers.Dense(10, activation="softmax")])

optimizer = keras.optimizers.SGD()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

n_epochs = 3
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,validation_data=(X_valid_scaled, y_valid))

Epoch 1/3
1719/1719 [==============================] - 9s 5ms/step - loss: 0.6819 - accuracy: 0.7666 - val_loss: 0.4215 - val_accuracy: 0.8512
Epoch 2/3
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4997 - accuracy: 0.8196 - val_loss: 0.3871 - val_accuracy: 0.8604
Epoch 3/3
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4604 - accuracy: 0.8317 - val_loss: 0.3807 - val_accuracy: 0.8660
